In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer # for imputing missing values
from sklearn.preprocessing import StandardScaler # for standardizing data
from csvToExcel import toExcel
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, LSTM # for creating regular densely-connected NN layers and RNN layers

In [112]:
# Function to check for missing data
def checkMissingData():
    df = pd.read_csv('DataFolder/data.csv')
    print(df.info())
    
checkMissingData()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38472 entries, 0 to 38471
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Entity                           38472 non-null  object 
 1   Continent                        38472 non-null  object 
 2   Latitude                         38472 non-null  float64
 3   Longitude                        38472 non-null  float64
 4   Average temperature per year     38472 non-null  int64  
 5   Hospital beds per 1000 people    38472 non-null  float64
 6   Medical doctors per 1000 people  38472 non-null  float64
 7   GDP/Capita                       38472 non-null  float64
 8   Population                       38472 non-null  int64  
 9   Median age                       38472 non-null  int64  
 10  Population aged 65 and over (%)  38472 non-null  int64  
 11  Date                             38472 non-null  object 
 12  Daily tests       

In [113]:
# Fill missing values with mean
def missingDataHandler():
    df = pd.read_csv('DataFolder/data.csv')
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    # From the missing data analysis, we know that the columns 'Daily tests', 'Cases', 'Deaths' have missing values
    missingColumns = ['Daily tests', 'Cases', 'Deaths']
    for column in missingColumns:
        df[column] = imputer.fit_transform(df[column].values.reshape(-1, 1)) # fit_transform() returns a numpy array, so we need to convert it back to a pandas dataframe
    print(df.info())
    return df
    
dataFrame = missingDataHandler()
# toExcel(dataFrame, 'preprocessedData')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38472 entries, 0 to 38471
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Entity                           38472 non-null  object 
 1   Continent                        38472 non-null  object 
 2   Latitude                         38472 non-null  float64
 3   Longitude                        38472 non-null  float64
 4   Average temperature per year     38472 non-null  int64  
 5   Hospital beds per 1000 people    38472 non-null  float64
 6   Medical doctors per 1000 people  38472 non-null  float64
 7   GDP/Capita                       38472 non-null  float64
 8   Population                       38472 non-null  int64  
 9   Median age                       38472 non-null  int64  
 10  Population aged 65 and over (%)  38472 non-null  int64  
 11  Date                             38472 non-null  object 
 12  Daily tests       

In [114]:
# User test date input
testDate = input('Enter a date to test the model (yyyy-mm-dd) after 2020-12-31: ')
testDate1 = pd.to_datetime(testDate)
testDate2 = testDate1 - pd.DateOffset(days = 9)
testDate2 = str(testDate2.date())
testDate1 = str(testDate1.date())

In [115]:
def getGreeceData(df):
    df = df[df['Entity'] == 'Greece']
    df = df[(df['Date'] >= testDate2) & (df['Date'] <= testDate1)]
    return df

dataFrame2 = getGreeceData(dataFrame)
# print(dataFrame2)

In [116]:
# Add new columns for positivity percent and daily cases
def addPositivityPercent(df):
    df['Daily Cases'] = abs(df['Cases'].diff())
    df['Positivity percent'] = df['Daily Cases'] / df['Daily tests']
    df['Date'] = pd.to_datetime(df['Date'])
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    df['Positivity percent'] = imputer.fit_transform(df['Positivity percent'].values.reshape(-1, 1))
    df['Daily Cases'] = imputer.fit_transform(df['Daily Cases'].values.reshape(-1, 1))
    return df

dataset = addPositivityPercent(dataFrame2)
# toExcel(dataset, 'GreeceData')
print(dataset.info())
# print(dataset)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 12103 to 12112
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Entity                           10 non-null     object        
 1   Continent                        10 non-null     object        
 2   Latitude                         10 non-null     float64       
 3   Longitude                        10 non-null     float64       
 4   Average temperature per year     10 non-null     int64         
 5   Hospital beds per 1000 people    10 non-null     float64       
 6   Medical doctors per 1000 people  10 non-null     float64       
 7   GDP/Capita                       10 non-null     float64       
 8   Population                       10 non-null     int64         
 9   Median age                       10 non-null     int64         
 10  Population aged 65 and over (%)  10 non-null     int64   

In [117]:
# Function to get all the data for Greece
def allGreeceData(df):
    df = df[df['Entity'] == 'Greece']
    df['Daily Cases'] = abs(df['Cases'].diff())
    df['Positivity percent'] = df['Daily Cases'] / df['Daily tests']
    df['Date'] = pd.to_datetime(df['Date'])
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    df['Positivity percent'] = imputer.fit_transform(df['Positivity percent'].values.reshape(-1, 1))
    df['Daily Cases'] = imputer.fit_transform(df['Daily Cases'].values.reshape(-1, 1))
    return df

dataFrame3 = allGreeceData(dataFrame)
print(dataFrame3.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369 entries, 11802 to 12170
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Entity                           369 non-null    object        
 1   Continent                        369 non-null    object        
 2   Latitude                         369 non-null    float64       
 3   Longitude                        369 non-null    float64       
 4   Average temperature per year     369 non-null    int64         
 5   Hospital beds per 1000 people    369 non-null    float64       
 6   Medical doctors per 1000 people  369 non-null    float64       
 7   GDP/Capita                       369 non-null    float64       
 8   Population                       369 non-null    int64         
 9   Median age                       369 non-null    int64         
 10  Population aged 65 and over (%)  369 non-null    int64  

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [118]:
# Here starts the training of the model
posPer = dataset['Positivity percent'].values
# print(posPer.shape)

In [119]:
# Standardize the data
sc_data = StandardScaler()
posPer = sc_data.fit_transform(posPer.reshape(-1, 1))

In [120]:
# Split the data into training and testing sets
sequence = posPer.shape[0] - 1
X = []
y = []
for i in range(sequence, len(posPer)):
    X.append(posPer[i-sequence:i])
    y.append(posPer[i])
X = np.array(X)
y = np.array(y)
# print(X.shape, y.shape)

In [121]:
# Prepare the model
model = Sequential()
model.add(LSTM(32, input_shape=(sequence, 1))) # LSTM layer with 32 neurons (LSTM = Long Short Term Memory) and input shape (number of time steps, number of features)
model.add(Dense(1)) # Dense layer with 1 neuron (output layer)
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error') # Adam optimizer and mean squared error loss function
# Fit keras model on the dataset
model.fit(X, y,  epochs=200, batch_size=1000) # 200 epochs is the number of times the model will cycle through the data and 1000 is the number of samples per gradient update

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 1.1426
Epoch 2/200
1/1 [==============================] - 0s 10ms/step - loss: 1.1108
Epoch 3/200
1/1 [==============================] - 0s 10ms/step - loss: 1.0794
Epoch 4/200
1/1 [==============================] - 0s 11ms/step - loss: 1.0484
Epoch 5/200
1/1 [==============================] - 0s 14ms/step - loss: 1.0176
Epoch 6/200
1/1 [==============================] - 0s 9ms/step - loss: 0.9871
Epoch 7/200
1/1 [==============================] - 0s 11ms/step - loss: 0.9568
Epoch 8/200
1/1 [==============================] - 0s 10ms/step - loss: 0.9266
Epoch 9/200
1/1 [==============================] - 0s 11ms/step - loss: 0.8964
Epoch 10/200
1/1 [==============================] - 0s 12ms/step - loss: 0.8661
Epoch 11/200
1/1 [==============================] - 0s 11ms/step - loss: 0.8358
Epoch 12/200
1/1 [==============================] - 0s 11ms/step - loss: 0.8053
Epoch 13/200
1/1 [==============================] - 

In [122]:
# Predict the positivity percent for the next 3 days
def predictNextDays(date):
    last_sequence = posPer[-sequence:]
    predictions = []
    for _ in range(3):
        input_sequence = last_sequence.reshape(1, sequence, 1) # reshape the input sequence to fit the model
        prediction = model.predict(input_sequence) # predict the positivity percent for the next day
        predictions.append(prediction[0]) # append the prediction to the predictions list
        last_sequence = np.concatenate((last_sequence[1:], prediction), axis=0) # concatenate is used to add the prediction to the last_sequence array 
    predictions = sc_data.inverse_transform(predictions) # inverse transform the predictions to get the actual values
    return predictions

predictions = predictNextDays(testDate1) # predict the positivity percent for the next 3 days

1/1 [==============================] - 0s 28ms/step


In [123]:
print("Predict positivity percent for the next 3 days:\n")
for i, prediction in enumerate(predictions):
    print(f"Date: {pd.to_datetime(testDate1) + pd.DateOffset(days = i)}")
    print(f"Prediction {i+1}: {round(prediction[0] * 100, 2)} %")
    print(f"Actual value: {round(dataFrame3[dataFrame3['Date'] == pd.to_datetime(testDate1) + pd.DateOffset(days = i)]['Positivity percent'].values[0] * 100, 2)} %\n")

Predict positivity percent for the next 3 days:

Date: 2021-01-01 00:00:00
Prediction 1: 2.1 %
Actual value: 2.08 %

Date: 2021-01-02 00:00:00
Prediction 2: 2.11 %
Actual value: 5.46 %

Date: 2021-01-03 00:00:00
Prediction 3: 2.07 %
Actual value: 4.59 %

